In [71]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
%matplotlib inline

traindata = pd.read_csv("train_values.csv",sep=',')
traindata.columns[traindata.isna().any()]
data=traindata
data.drop(['building_id'], axis=1, inplace=True)
trainlabels = pd.read_csv("train_labels.csv",sep=',')
trainout=trainlabels
trainout.drop(['building_id'], axis=1, inplace=True)
testvalues = pd.read_csv("test_values.csv",sep=',')
testdata=testvalues
testdata.drop(['building_id'], axis=1, inplace=True)

In [47]:
cleanup_letters = {"land_surface_condition": {"n": 1, "o": 2, "t":3},
                "foundation_type": {"h": 1, "i": 2, "r": 3, "u": 4, "w": 5},               
                "roof_type": {"n": 1, "q": 2, "x": 3},
                "ground_floor_type": {"f": 1, "m": 2, "v": 3, "x": 4, "z": 5},
                "other_floor_type": {"j": 1, "q": 2, "s": 3, "x": 4},
                "position": {"j": 1, "o": 2, "s": 3, "t": 4},
                "plan_configuration": {"a": 1, "c": 2, "d": 3, "f": 4, "m": 5, "n": 6, "o": 7, "q": 8, "s": 9, "u":10},
                "legal_ownership_status": {"a": 1, "r": 2, "v": 3, "w": 4}             
               }
data.replace(cleanup_letters, inplace=True)
testdata.replace(cleanup_letters, inplace=True)
a = list(data)
del a[-1]
x = data[a].values
y = trainout['damage_grade'].values

In [73]:
test = testdata[a].values

In [48]:
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [39]:
(batch_size, timesteps) = X_train.shape
# This is done because the RNN class in Keras expects a 
# 3D Tensor
X_train = np.reshape(X_train, (batch_size, timesteps, 1))

In [49]:
(batch_size, timesteps) = x.shape
# This is done because the RNN class in Keras expects a 
# 3D Tensor
x = np.reshape(x, (batch_size, timesteps, 1))

In [50]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [51]:
regressor = Sequential()

# units is the output dimensionality
# return sequences will return the sequence
# which will be required to the next LSTM 

# input shape will need only the last 2 dimensions
# of your input
################# 1st layer #######################
regressor.add(LSTM(units=50, return_sequences=True, 
                   input_shape=(timesteps, 1)))

# add Dropout to do regulariztion
# standard practise to use 20%
regressor.add(Dropout(0.2))

################# 2nd layer ######################
# After the first time, it's not required to 
# specify the input_shape

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

################# 3rd layer ######################
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
              
################# 4th layer ######################
# the last layer needn't return the sequence, so
# return_sequences will be False
              
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

################# 5th layer ######################
# Last layer would be the fully connected layer,
# or the Dense layer
#
# The last word will predict a single number
# hence units=1

regressor.add(Dense(units=1))
              
# Compiling the RNN
# The loss function for classification problem is 
# cross entropy, since this is a regression problem
# the loss function will be mean squared error
              
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [52]:
regressor.fit(x, y, epochs=2, batch_size=1024)

Epoch 1/2
260601/260601 [==============================] - 511s 2ms/step - loss: 0.5428
Epoch 2/2
260601/260601 [==============================] - 530s 2ms/step - loss: 0.4030


In [ ]:
# Persisting the model for future use
from keras.models import load_model
# Save
regressor.save('regressor.hd5')
# Load
# model = load_model('regressor.hd5')

In [57]:
# Reshape : convert our 2D array to 3D
(batch_size, timesteps) = X_test.shape
# This is done because the RNN class in Keras expects a 
# 3D Tensor
X_test = np.reshape(X_test, (batch_size, timesteps, 1))

In [65]:
X_test.shape

(52121, 37, 1)

In [58]:
# Making prediction
y_pred = regressor.predict(X_test)
# inverse the scaling
#predict = sc.inverse_transform(predicted_stock_price)
y_pred = y_pred.astype(int)
y_pred

array([[2],
       [2],
       [2],
       ...,
       [2],
       [2],
       [2]])

In [35]:
f1_model=f1_score(y_test,y_pred,average='weighted',labels=np.unique(y_pred))
f1_model

0.7260217051231913

In [74]:
# Reshape : convert our 2D array to 3D
(batch_size, timesteps) = test.shape
# This is done because the RNN class in Keras expects a 
# 3D Tensor
testdata = np.reshape(test, (batch_size, timesteps, 1))

In [76]:
# Making prediction
y_pred = regressor.predict(testdata)
# inverse the scaling
#predict = sc.inverse_transform(predicted_stock_price)
y_pred = y_pred.astype(int)
y_pred

ValueError: could not convert string to float: 't'

In [ ]:
#df = pd.DataFrame({ 'damage_grade': np.array([0, 1, 2, 3]) })
out = pd.DataFrame({'damage_grade':data[:,0]})